* [前言&資料預處理](#前言&資料預處理)
* [特徵工程&找尋對店家有用的資訊](#特徵工程&找尋對店家有用的資訊)
* [卡方檢定&特色飲料客群的特徵](#卡方檢定&特色飲料客群的特徵)
* [小結](#小結)

 <a name="前言&資料預處理"></a>
## 前言&資料預處理

In [1]:
# 我最近回去看了大學做的專題，覺得過去有些統計內容沒什麼統計意義，像是這個：
# 在性別方面與等待時間做交叉列表和卡方檢定
# 卡方檢定的假設：
# H0：等待時間和性別獨立
# Ha：等待時間和性別不獨立

# 即便顧客願意等待的時間與性別不獨立，假如女姓願意等比較久，店家也不可能由於此便先做男性的餐點，所以我覺得這對店家是沒什麼意義的資訊
# 所以我想以目前所學的專業，再做一次大學專題看看

# 問卷內容: https://docs.google.com/document/d/1iRGezP8tFIvxEyNn64fMnIX6W8hRYuZrAacuYqF3V6I/edit?usp=sharing
# 專題PDF: https://drive.google.com/file/d/1PA3fPIAzuRSKE5UZoWLe27fAYS2VUaWh/view?usp=sharing

In [2]:
import os
import pandas as pd
import numpy as np
import datetime
import glob
from datetime import date
import warnings
warnings.filterwarnings("ignore")  
pd.set_option('display.max_columns', None)
df=pd.read_excel(r'often/問卷譯碼本.xlsx',sheet_name='公版')
print(df.shape)
df.head(2)

(500, 103)


,流水號,訪員編號,受訪地點,受訪時間,同遊人數(含受訪者),受訪者性別,1_1,1_2,1_3,1_4,1_5,1_6,1_7,1_8,1_9,1_10,1_11,1_12,1_13,1_14,2_1_a,2_1_b,2_1_c,2_1_d,2_1_e,2_2_1,2_2_2,2_2_3,2_2_4,2_2_5,2_2_6,2_2_7,2_2_8,2_2_8_1,2_3,2_4_1,2_4_2,2_4_3,2_4_4,2_4_5,2_4_6,2_4_7,2_4_8,2_4_9,2_4_10,2_4_11,2_4_12,2_4_13,2_4_14,2_4_15,2_4_16,2_4_17,2_4_18,2_4_19,2_4_20,2_4_21,2_4_22,2_4_22_1,2_5,2_6_1,2_6_2,2_6_3,2_6_4,2_6_5,2_6_6,2_6_7,2_6_8,2_6_9,2_6_9_1,2_7,2_8,2_9_1,2_9_2,2_9_3,2_9_4,2_9_5,2_9_6,2_9_7,2_9_8,2_9_9,2_9_10,2_9_10_1,3_1_1,3_1_2,3_1_3,3_1_4,3_1_5,3_1_6,3_2,3_3,3_4,3_4_1,3_5,3_6,3_7_1,3_7_2,3_7_2_9,3_7_2_10,3_8,3_8_1,3_9_1,3_9_2,3_10
0,1,2,1,20181122,1,0,5,5,1,4,4,5.0,4,1,4,4,5,3,5,4,5,4,4,5,4,1,1,0,0,0,0,0,0,888,2,0,0,1,1,1,0,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,888,3,0,0,1,0,0,0,0,0,0,888,2,1,1,0,0,0,0,0,0,0,0,0,888,0,0,1,0,0,0,6,1,1,888,2,1,1,1,888,888,1,888,1,975065,1
1,2,2,1,20181122,5,1,5,5,1,3,3,4.0,3,2,5,4,4,4,4,5,5,5,5,5,5,1,0,0,1,0,0,0,0,888,4,0,0,0,1,0,0,0,1,1,1,0,0,1,1,0,1,1,0,1,1,0,0,888,10,0,0,0,0,0,0,0,0,1,清心福全,1,1,1,0,0,1,0,0,0,0,0,0,888,1,0,0,0,0,0,7,1,3,888,5,2,1,6,888,888,3,888,1,933618,1


In [3]:
# 註：999-表該題漏答
# 註：888-表該題不適用
# 註：777-表該題無效
# 先來處以缺失值，以我的理解，999 888 777全都為視為缺失值，不需要特別分類
# 而由於只有500筆問卷資料，所以我傾向於幫缺失值補值，而非刪除有缺失值的列
# 其中欄位1_1到2_1全為受訪者填寫，且全為詢問滿意度的問題，6為最滿意，1為最不滿意
# 大部分的問卷滿意度設為1到5，但是若設1-5，很多人會傾向於全部填3，也就是對方懶的填或者沒意見，那一般來說參考價值便會降低
# 當初設為1到6為了避免都填3(中立、沒意見)的資料

df2=df.copy()
df2=df2.astype(str)

for i in range(df2.shape[1]):
    df2[df2.columns[i]]=df2[df2.columns[i]].str.strip()
    df2[df2.columns[i]]=df2[df2.columns[i]].str.replace('999','')
    df2[df2.columns[i]]=df2[df2.columns[i]].str.replace('888','')
    df2[df2.columns[i]]=df2[df2.columns[i]].str.replace('777','')
    df2[df2.columns[i]]=df2[df2.columns[i]].str.replace('.0','')
    df2[df2.columns[i]]=df2[df2.columns[i]].str.replace('nan','')
df2=df2.replace('',np.nan)
df2['1_6'].value_counts()

KeyboardInterrupt: 

In [ ]:
# 欄位2_5無法轉換為數字格式，發現包含no，以題目敘述，填寫no表示以上店家都不推薦，因此轉換為空值
# 4. 請勾選您在逢甲商圈曾經嘗試過的飲料店? (可複選) 
# 1.□ 金享 2.□ 緣棧 3.□ 大苑子 4.□ 五十嵐 5.□ 甲文青
# 6.□ 幸福堂 7.□ 珍煮丹 8.□ 茶湯會 9.□ 斜角巷 10.□ 喫茶趣
# 11.□ 蔡大明 12.□ 鮮果匠 13.□ 鮮茶道 14.□ Double A 15.□ 良辰吉時
# 16.□ 清心福全 17.□ 圓石禪飲 18.□ 一芳水果茶 19.□ 手作功夫茶
# 20.□ 可不可熟成紅茶 21.□ 超好喝木瓜牛奶 22. □ 其他____________
# 5. 呈上題，您最推薦的飲料店家是(請填上述 1 個編號) __________

df2['2_5'].value_counts()

4     58
3     55
7     34
21    32
18    30
8     29
17    28
22    28
16    26
11    21
1     16
9     16
6     12
13     8
2      6
14     6
19     4
5      4
12     3
15     3
no     2
Name: 2_5, dtype: int64

In [ ]:
df2['2_5']=df2['2_5'].replace('no',np.nan)

In [ ]:
# 部分欄位為「其他」的填空題，不適合放在一起統計，所以先刪除
# 如欄位3_4_1為這題4其他的填寫處
# 您目前的居住地(非戶籍地)是哪個縣市？
# 1.□ 台中市 2.□ 彰化縣 3.□ 苗栗、南投、雲林、嘉義地區 4.□ 其他
df2['3_4_1'].value_counts()

台北      22
台南       9
高雄       9
新北       8
桃園       7
新竹       5
馬來西亞     3
香港       3
屏東       2
澳門       1
宜蘭       1
日本       1
中壢       1
新加坡      1
Name: 3_4_1, dtype: int64

In [ ]:
# 找出空值多於一半的欄位並刪除
df2.columns[df2.isna().sum() > len(df2) / 2]

Index(['2_2_8_1', '2_4_22_1', '2_6_9_1', '2_9_10_1', '3_4_1', '3_7_2',
       '3_7_2_9', '3_7_2_10', '3_8_1', '3_9_2'],
      dtype='object')

In [ ]:
df3=df2.drop(columns=['2_2_8_1', '2_4_22_1', '2_6_9_1', '2_9_10_1', '3_4_1', '3_7_2',
       '3_7_2_9', '3_7_2_10', '3_8_1', '3_9_2'])

# 由於欄位都是類別型資料，所以缺失值都補眾數
# df3 = df3.replace('',(df.mode().iloc[0]))
df3 = df3.fillna(df.mode().iloc[0])

for i in range(df3.shape[1]):
    df3[df3.columns[i]]=df3[df3.columns[i]].str.replace('.0','')
# 確認是否還有缺失值 (false無缺失值)    
df3 = df3.fillna(df.mode().iloc[0])
df3.isna().any().any()
# df3['1_6'].value_counts()

False

 <a name="特徵工程&找尋對店家有用的資訊"></a>
## 特徵工程&找尋對店家有用的資訊

In [ ]:
# 假設我想在在逢甲商圈開設飲料店，並且想從資料中獲取有用資訊，那我會想先分析以下的問題，看要走加盟店或是特色飲料店
# 3. 您在逢甲商圈偏好在連鎖飲料店或逢甲特色飲料店消費?
# 1.□ 只願意購買特色飲料店 2.□ 偏好特色飲料店 3.□ 都願意
# 4.□ 偏好連鎖飲料店 5.□ 只願意購買連鎖飲料店 6.□ 沒有喝飲料的習慣
df3['2_3'].value_counts()

3      232
2      148
4       48
1       29
6       18
5       15
3.0     10
Name: 2_3, dtype: int64

In [ ]:
# 發現3.0應該要跟3合併，因此把.0排除掉
df3=df3.astype(str)
for i in range(df3.shape[1]):
    # df3[df3.columns[i]].str.replace('.0','')
    df3[df3.columns[i]] = df3[df3.columns[i]].str.replace('.0','')
df3['2_3'].value_counts()

3    242
2    148
4     48
1     29
6     18
5     15
Name: 2_3, dtype: int64

In [ ]:
# 接著來看偏好特色飲料店的人，他們有甚麼特徵，與非偏好特色飲料店的人有什麼區別
df4=df3[(df3['2_3'] == '1') | (df3['2_3'] == '2')]
df5=df3[~(df3['2_3'] == '1') & ~(df3['2_3'] == '2')]

 <a name="卡方檢定&特色飲料客群的特徵"></a>
## 卡方檢定&特色飲料客群的特徵

In [ ]:
# 2. 您這次在逢甲商圈會購買下列哪一類型的飲料? (可複選)
#  1.□ 茶類 2.□ 鮮奶類(ex:奶茶、紅茶拿鐵) 
#  3.□ 鮮果類 4.□ 調酒類 5.□ 氣泡飲 
#  6.□ 冰沙 7.□ 咖啡 8.□ 其他____________

#偏好特色飲料店的人較不愛購買茶類(50.85% VS 65.02%)，且有顯著差距(P < 0.05)
from scipy.stats import chisquare
def chi(df, df2, column_name, percent):
    df['percent'] = df[column_name] / df[column_name].sum()
    print(df)

    df2['percent'] = df2[column_name] / df2[column_name].sum()
    print(df2)

    observed_values_normalized = df[percent].values  * np.sum(df2[column_name].values)
    # observed_values_normalized = df[percent].values  * df2[column_name].shape[0]
    expected_values_normalized = df2[column_name].values 

    result = chisquare(f_obs=observed_values_normalized, f_exp=expected_values_normalized)
    # result = chisquare(f_obs=df['2_2_1'], f_exp=df2['2_2_1'])
    print("Chi-Square statistic:", result.statistic)
    print("P-value:", result.pvalue)

tea = df4['2_2_1'].value_counts().reset_index()
tea2 = df5['2_2_1'].value_counts().reset_index()

chi(tea, tea2, '2_2_1', 'percent')

  index  2_2_1   percent
0     1     90  0.508475
1     0     87  0.491525
  index  2_2_1   percent
0     1    210  0.650155
1     0    113  0.349845
Chi-Square statistic: 28.50547545261962
P-value: 9.3433884267113e-08


In [ ]:
#偏好特色飲料店的人較愛購買鮮奶類，但無顯著差距
milk = df4['2_2_2'].value_counts().reset_index()
milk2 = df5['2_2_2'].value_counts().reset_index()
chi(milk, milk2, '2_2_2', 'percent')

  index  2_2_2   percent
0     1    117  0.661017
1     0     60  0.338983
  index  2_2_2   percent
0     1    200  0.619195
1     0    123  0.380805
Chi-Square statistic: 2.3959626006917967
P-value: 0.12164884889301604


In [ ]:
#偏好特色飲料店的人較不愛購買鮮果類，且有顯著差距
juice = df4['2_2_3'].value_counts().reset_index()
juice2 = df5['2_2_3'].value_counts().reset_index()
chi(juice, juice2, '2_2_3', 'percent')

  index  2_2_3   percent
0     0    127  0.717514
1     1     50  0.282486
  index  2_2_3   percent
0     0    203  0.628483
1     1    120  0.371517
Chi-Square statistic: 10.965142255026262
P-value: 0.00092841819416956


In [ ]:
#偏好特色飲料店的人較愛購買調酒類，且有顯著差距
alcohol = df4['2_2_4'].value_counts().reset_index()
alcohol2 = df5['2_2_4'].value_counts().reset_index()
chi(alcohol, alcohol2, '2_2_4', 'percent')

  index  2_2_4   percent
0     0    152  0.858757
1     1     25  0.141243
  index  2_2_4   percent
0     0    299  0.925697
1     1     24  0.074303
Chi-Square statistic: 21.042168016867635
P-value: 4.49285577760626e-06


In [ ]:
#偏好特色飲料店的人較愛購買氣泡飲，但無顯著差距
soda = df4['2_2_5'].value_counts().reset_index()
soda2 = df5['2_2_5'].value_counts().reset_index()
chi(soda, soda2, '2_2_5', 'percent')

  index  2_2_5   percent
0     0    163  0.920904
1     1     14  0.079096
  index  2_2_5   percent
0     0    305  0.944272
1     1     18  0.055728
Chi-Square statistic: 3.3519425108262597
P-value: 0.0671257570717653


In [ ]:
#偏好特色飲料店的人較愛購買冰沙，但無顯著差距
ice = df4['2_2_6'].value_counts().reset_index()
ice2 = df5['2_2_6'].value_counts().reset_index()
chi(ice, ice2, '2_2_6', 'percent')

  index  2_2_6   percent
0     0    153  0.864407
1     1     24  0.135593
  index  2_2_6   percent
0     0    286  0.885449
1     1     37  0.114551
Chi-Square statistic: 1.4100012156605088
P-value: 0.23505619143291318


In [ ]:
#偏好特色飲料店的人較愛購買咖啡，但無顯著差距
coffie = df4['2_2_7'].value_counts().reset_index()
coffie2 = df5['2_2_7'].value_counts().reset_index()
chi(coffie, coffie2, '2_2_7', 'percent')

  index  2_2_7   percent
0     0    160  0.903955
1     1     17  0.096045
  index  2_2_7   percent
0     0    295  0.913313
1     1     28  0.086687
Chi-Square statistic: 0.3572592698732661
P-value: 0.5500323208837616


In [ ]:
# 特色飲料客群較喜愛購買調酒類，且有顯著差距
# 特色飲料客群較喜愛購買鮮奶類、氣泡飲、冰沙、咖啡，但無顯著差距

# 7. 您通常在逢甲商圈購買一杯飲料的金額為何？
# 1：35元以下；2：36-50元；3：51-65元；4：66-80元；5：81元以上
# 特色飲料客群通常買一杯飲料花36-65元(107年的資訊)，且願意花的金額比非特色飲料客群高，且有顯著差距
price = df4['2_7'].value_counts().reset_index()
price2 = df5['2_7'].value_counts().reset_index()
chi(price, price2, '2_7', 'percent')

  index  2_7   percent
0     2   84  0.474576
1     3   70  0.395480
2     4   12  0.067797
3     5    7  0.039548
4     1    4  0.022599
  index  2_7   percent
0     2  171  0.529412
1     3  112  0.346749
2     1   25  0.077399
3     4   12  0.037152
4     5    3  0.009288
Chi-Square statistic: 10.643084894719482
P-value: 0.03088207962105372


In [ ]:
# 我認為以上的例子用chi-square去算有些不妥，因為假如實驗組是35元以下的有200人，剩下都0人
# 對照組是36-50元的200人，剩下都0人，那一定有顯著差距，但是都是偏低價位，對商家來說不太算是顯著差距
# 所以我把50元以下分一群(index 1)，51元以上分一群(index 3)
# 分群發現特色飲料客群通常買一杯飲料願意花的金額比非特色飲料客群高，且有顯著差距

df4['2_72']=df4['2_7']
df4['2_72']=df4['2_72'].astype(str)
df4['2_72']=df4['2_72'].str.replace('2','1')
df4['2_72']=df4['2_72'].str.replace('4','3')
df4['2_72']=df4['2_72'].str.replace('5','3')

df5['2_72']=df5['2_7']
df5['2_72']=df5['2_72'].astype(str)
df5['2_72']=df5['2_72'].str.replace('2','1')
df5['2_72']=df5['2_72'].str.replace('4','3')
df5['2_72']=df5['2_72'].str.replace('5','3')
price3 = df4['2_72'].value_counts().reset_index()
price4 = df5['2_72'].value_counts().reset_index()
chi(price3, price4, '2_72', 'percent')


  index  2_72   percent
0     3    89  0.502825
1     1    88  0.497175
  index  2_72   percent
0     1   196  0.606811
1     3   127  0.393189
Chi-Square statistic: 14.63861251733282
P-value: 0.00013021903645688937


In [ ]:
# 2_8您購買飲料時，願意花多少時間等待? 
# 1：5分鐘以內；2：6-10分鐘；3：11-15分鐘；4：16-20分鐘；5：21分鐘以上
wait = df4['2_8'].value_counts().reset_index()
wait2 = df5['2_8'].value_counts().reset_index()
chi(wait, wait2, '2_8', 'percent')

  index  2_8   percent
0     2   86  0.485876
1     1   63  0.355932
2     3   22  0.124294
3     5    3  0.016949
4     4    3  0.016949
  index  2_8   percent
0     2  154  0.476780
1     1  138  0.427245
2     3   26  0.080495
3     5    4  0.012384
4     4    1  0.003096
Chi-Square statistic: 32.16359655794538
P-value: 1.771321268459057e-06


In [ ]:
# 同樣把等待時間分為兩群，10分鐘以下一群(index 1)，11分鐘以上一群(index 3)
# 特色飲料客群較願意等，且有顯著差距，不過仍然只有15.82%的客人願意等11分鐘以上
df4['2_82']=df4['2_8']
df4['2_82']=df4['2_82'].astype(str)
df4['2_82']=df4['2_82'].str.replace('2','1')
df4['2_82']=df4['2_82'].str.replace('4','3')
df4['2_82']=df4['2_82'].str.replace('5','3')

df5['2_82']=df5['2_8']
df5['2_82']=df5['2_82'].astype(str)
df5['2_82']=df5['2_82'].str.replace('2','1')
df5['2_82']=df5['2_82'].str.replace('4','3')
df5['2_82']=df5['2_82'].str.replace('5','3')

wait3 = df4['2_82'].value_counts().reset_index()
wait4 = df5['2_82'].value_counts().reset_index()
chi(wait3, wait4, '2_82', 'percent')

  index  2_82   percent
0     1   149  0.841808
1     3    28  0.158192
  index  2_82   percent
0     1   292  0.904025
1     3    31  0.095975
Chi-Square statistic: 14.410504145652366
P-value: 0.00014698016195958564


In [ ]:
# 5. 您的年齡為何？
# 1.□ 15 歲(含)以下 2.□ 16~18 歲 3.□ 19~23 歲 4.□ 24~28 歲
# 5.□ 29~34 歲 6.□ 35 歲以上
# 特色飲料客群年齡大多在19-28歲，年齡層與非特色飲料客群很接近
age = df4['3_5'].value_counts().reset_index()
age2 = df5['3_5'].value_counts().reset_index()
chi(age, age2, '3_5', 'percent')

  index  3_5   percent
0     3   87  0.491525
1     4   34  0.192090
2     2   23  0.129944
3     6   22  0.124294
4     5    9  0.050847
5     1    2  0.011299
  index  3_5   percent
0     3  162  0.501548
1     4   62  0.191950
2     2   34  0.105263
3     5   32  0.099071
4     6   26  0.080495
5     1    7  0.021672
Chi-Square statistic: 9.13852700205304
P-value: 0.10366472348479981


In [ ]:
# 2. 您這次到逢甲商圈的交通工具為何？
# 1.□ 徒步 2.□ 腳踏車 3.□ 機車 4.□ 汽車
# 5.□ 計程車 6.□ 公車 7.□ 遊覽車
# 兩種客群的交通工具接近
traffic = df4['3_2'].value_counts().reset_index()
traffic2 = df5['3_2'].value_counts().reset_index()
chi(traffic, traffic2, '3_2', 'percent')


  index  3_2   percent
0     3   67  0.378531
1     1   45  0.254237
2     4   32  0.180791
3     6   21  0.118644
4     7    5  0.028249
5     5    5  0.028249
6     2    2  0.011299
  index  3_2   percent
0     3  131  0.405573
1     1   64  0.198142
2     4   63  0.195046
3     6   44  0.136223
4     5   12  0.037152
5     7    8  0.024768
6     2    1  0.003096
Chi-Square statistic: 14.649222824485122
P-value: 0.02316763976706411


In [ ]:
a=df4[df4.columns[5]].value_counts().reset_index()
a['percent'] = a[a.columns[1]] / a[a.columns[1]].sum()
b=df5[df5.columns[5]].value_counts().reset_index()

observed_values_normalized = a['percent'].values  * df5.shape[0]
expected_values_normalized = b[b.columns[1]].values 

result = chisquare(f_obs=observed_values_normalized, f_exp=expected_values_normalized)
print(a)
print(b)
print("Chi-Square statistic:", result.statistic)
print("P-value:", result.pvalue)

  index  受訪者性別   percent
0     1    104  0.587571
1     0     73  0.412429
  index  受訪者性別
0     1    167
1     0    156
Chi-Square statistic: 6.436819992358873
P-value: 0.011177857087388457


In [ ]:
# import pandas as pd

# # 創建或載入您的 DataFrame（將此替換為實際數據）
# data = {
#     'column1': [1, 2, 3],
#     'column2': [4, 5, 6],
#     'column3': [7, 8, 9]
# }
# df4 = pd.DataFrame(data)

# # 創建新的列，其列名基於現有的列名
# new_column_name = str(df4.columns[0]) + '2'
# df4[new_column_name] = '1'

# print(df4)


In [ ]:
# 尋找特色飲料客群與其他客群，哪些類別有顯著差距
# 題目1_1到2_1都是6最滿意，1最不滿意的類型，我認為要看顯著差距，也同樣只先分為兩類，滿意與不滿意較佳
for i in range(6,30):
    df4[df4.columns[i]+'q'] = df4[df4.columns[i]]
    df4[df4.columns[i]+'q']=df4[df4.columns[i]+'q'].str.replace('3','1')
    df4[df4.columns[i]+'q']=df4[df4.columns[i]+'q'].str.replace('2','1')
    df4[df4.columns[i]+'q']=df4[df4.columns[i]+'q'].str.replace('6','4')
    df4[df4.columns[i]+'q']=df4[df4.columns[i]+'q'].str.replace('5','4')
    df4[df4.columns[i]+'q']=df4[df4.columns[i]+'q'].str.replace('6','4')
    
    df5[df5.columns[i]+'q'] = df5[df5.columns[i]]
    df5[df5.columns[i]+'q']=df5[df5.columns[i]+'q'].str.replace('3','1')
    df5[df5.columns[i]+'q']=df5[df5.columns[i]+'q'].str.replace('2','1')
    df5[df5.columns[i]+'q']=df5[df5.columns[i]+'q'].str.replace('6','4')
    df5[df5.columns[i]+'q']=df5[df5.columns[i]+'q'].str.replace('5','4')
    df5[df5.columns[i]+'q']=df5[df5.columns[i]+'q'].str.replace('6','4')
# df4.columns[6] + '2'
# df4.columns[6]

In [ ]:
df42=df4[['1_1q', '1_2q',
       '1_3q', '1_4q', '1_5q', '1_6q', '1_7q', '1_8q', '1_9q', '1_10q', '1_11q', '1_12q',
       '1_13q', '1_14q', '2_1_aq', '2_1_bq', '2_1_cq', '2_1_dq', '2_1_eq']]

df52=df5[['1_1q', '1_2q',
       '1_3q', '1_4q', '1_5q', '1_6q', '1_7q', '1_8q', '1_9q', '1_10q', '1_11q', '1_12q',
       '1_13q', '1_14q', '2_1_aq', '2_1_bq', '2_1_cq', '2_1_dq', '2_1_eq']]

In [ ]:
for i in range(df42.shape[1]):
    a=df42[df42.columns[i]].value_counts().reset_index()
    a['percent'] = a[a.columns[1]] / a[a.columns[1]].sum()
    b=df52[df52.columns[i]].value_counts().reset_index()
    b['percent'] = b[b.columns[1]] / b[a.columns[1]].sum()
    
    observed_values_normalized = a['percent'].values  * df52.shape[0]
    expected_values_normalized = b[b.columns[1]].values 

    result = chisquare(f_obs=observed_values_normalized, f_exp=expected_values_normalized)

    if result.pvalue < 0.05:
        print(df42.columns[i],)
        print(a)
        print(b)
        print("Chi-Square statistic:", result.statistic)
        print("P-value:", result.pvalue,'\n')

1_1q
  index  1_1q   percent
0     4   154  0.870056
1     1    23  0.129944
  index  1_1q   percent
0     4   257  0.795666
1     1    66  0.204334
Chi-Square statistic: 10.994353487212997
P-value: 0.000913898867995581 

1_3q
  index  1_3q   percent
0     4   127  0.717514
1     1    50  0.282486
  index  1_3q   percent
0     4   195  0.603715
1     1   128  0.396285
Chi-Square statistic: 17.4839488184127
P-value: 2.897436153673869e-05 

1_6q
  index  1_6q   percent
0     4   133  0.751412
1     1    44  0.248588
  index  1_6q   percent
0     4   258  0.798762
1     1    65  0.201238
Chi-Square statistic: 4.505048165131811
P-value: 0.0337949444128801 

1_8q
  index  1_8q   percent
0     4   118  0.666667
1     1    59  0.333333
  index  1_8q   percent
0     4   192  0.594427
1     1   131  0.405573
Chi-Square statistic: 6.99171260955612
P-value: 0.008188796420447843 

1_14q
  index  1_14q   percent
0     4    148  0.836158
1     1     29  0.163842
  index  1_14q   percent
0     4    2

In [ ]:
# 以下資訊有顯著差異，特色飲料客群：
# 較同意逢甲商圈的商品有獨特性(1_1 87.01% VS 79.57%)
# 較同意逢甲商圈的商品較格低廉(1_3)
# 較同意逢甲商圈的食物便宜(1_8)
# 較在意口味的獨特性和包裝的獨特性(2_1)

# 較不同意逢甲商圈的的商家態度親切(1_6)
# 較無再次來逢甲商圈消費的意願(1_14)

In [ ]:
# 找尋特色飲料特群較喜愛哪些行銷手法
df43=df4[['2_9_1', '2_9_2', '2_9_3', '2_9_4', '2_9_5', '2_9_6', '2_9_7', '2_9_8', '2_9_9', '2_9_10']]
df53=df5[['2_9_1', '2_9_2', '2_9_3', '2_9_4', '2_9_5', '2_9_6', '2_9_7', '2_9_8', '2_9_9', '2_9_10']]

for i in range(df43.shape[1]):
    a=df43[df43.columns[i]].value_counts().reset_index()
    a['percent'] = a[a.columns[1]] / a[a.columns[1]].sum()
    b=df53[df53.columns[i]].value_counts().reset_index()
    b['percent'] = b[b.columns[1]] / b[a.columns[1]].sum()
    
    observed_values_normalized = a['percent'].values  * df53.shape[0]
    expected_values_normalized = b[b.columns[1]].values 

    result = chisquare(f_obs=observed_values_normalized, f_exp=expected_values_normalized)

    if result.pvalue < 0.05:
        print(df43.columns[i],)
        print(a)
        print(b)
        print("Chi-Square statistic:", result.statistic)
        print("P-value:", result.pvalue,'\n')

2_9_2
  index  2_9_2   percent
0     0    135  0.762712
1     1     42  0.237288
  index  2_9_2  percent
0     0    261  0.80805
1     1     62  0.19195
Chi-Square statistic: 4.280492960514399
P-value: 0.03855216565308863 

2_9_4
  index  2_9_4   percent
0     0    166  0.937853
1     1     11  0.062147
  index  2_9_4   percent
0     0    287  0.888545
1     1     36  0.111455
Chi-Square statistic: 7.929791050747909
P-value: 0.004862741215178248 

2_9_6
  index  2_9_6   percent
0     0    128  0.723164
1     1     49  0.276836
  index  2_9_6   percent
0     0    250  0.773994
1     1     73  0.226006
Chi-Square statistic: 4.770724534185801
P-value: 0.028947640785074558 

2_9_9
  index  2_9_9   percent
0     0    170  0.960452
1     1      7  0.039548
  index  2_9_9   percent
0     0    299  0.925697
1     1     24  0.074303
Chi-Square statistic: 5.672431940438074
P-value: 0.017233548479609662 



In [ ]:
# 特色飲料客群較喜愛：2_9_2每日限量、2_9_6試飲
# 特色飲料客群較不喜愛：2_9_4會員卡優惠、2_9_9app抽獎優惠
# 沒有顯著差距：2_9_1買一送一、2_9_3自帶環保杯折價、2_9_5打卡優惠、2_9_7一日店長、2_9_8部落客推薦、2_9_10其他

 <a name="小結"></a>
## 小結
數據分析有很多需要注重的思考點，這份數據分析的資料來源是請路人填問卷，也就是抽樣調查，但填問卷方式蒐集到的資料很可能往往不能代表母體，因為願意填問卷的人通常是比較熱心或是不善常拒絕的人，而願意填問卷與不願意填問卷的群體，很可能在某些方面也有顯著差距，加上這些是107年的資訊，因此很可能不適用於現今情況

若是數據分析的內容更需要精確性，蒐集樣本的方式也需要審慎考慮，像是用抽獎問卷，若有漏填則失去資格等等，讓大部分的群體都願意認真填問卷